In [44]:
%matplotlib inline
import d2lzh as d2l #d2l做了一些为了方便的外部封装
from mxnet import autograd,gluon,init,nd  #mxnet里边放了很多数学工具
from mxnet.gluon  import data as gdata,loss as gloss,nn #数据读取，损失函数，和层gluon里边放的东西是最重要的

In [45]:
'''简洁实现'''

'''提供一下训练数据'''
#为了让出现过拟合的情况，训练数据给的特别少
n_train,n_test,num_inputs = 20 ,100 ,200
#设置权重和偏差,nd.ones返回数据都为1的数组
true_w,true_b = nd.ones((num_inputs,1))*0.01 ,0.05
#创建样本和标签
features = nd.random.normal(shape=(n_train+n_test,num_inputs))
#把labels组装出来，再加上噪音
labels = nd.dot(features,true_w) + true_b;
labels += nd.random.normal(scale=0.01,shape=labels.shape)
#最后训练和测试截取的是行的部分
train_features,test_features = features[:n_train,:],features[n_train:,:]
train_labels,test_labels = labels[:n_train],labels[n_train:]

'''读取数据集'''
batch_size,num_epochs,lr = 1,100,0.003 
#读取数据集
train_iter = gdata.DataLoader(gdata.ArrayDataset(
train_features, train_labels), batch_size, shuffle=True)

'''损失函数'''
loss = d2l.squared_loss


'''其他都放在了这里边'''
def fit_and_plot_gluon(wd):
    #串联各个层的容器
    net = nn.Sequential()
    #添加层
    net.add(nn.Dense(1))
    #初始化模型参数
    net.initialize(init.Normal(sigma=1))
    #对权重衰减，权重名称一般是以weight结尾
    trainer_w = gluon.Trainer(net.collect_params('.*weight'),'sgd',{'learning_rate':0.003,'wd':wd})
    #不对偏差参数衰减，偏差名称一般以bias结尾
    trainer_b = gluon.Trainer(net.collect_params('.*bias'),'sgd',{'learning_rate':0.003})
        
    train_ls,test_ls = [],[]
    for _ in range(num_epochs):
        #在这里进行w和b的优化
        for X,y in train_iter:
            with autograd.record():
                l = loss(net(X),y)
            l.backward()
            #对两个Trainer实例分别调用step函数，从而分别 更新权重分偏差
            #之前都是手动sgd，现在变成了step
            trainer_w.step(batch_size)
            trainer_b.step(batch_size)
        #前边计算过了之后，我们就可以添加训练过程中的loss数据了
        train_ls.append(loss(net(train_features),train_labels).mean().asscalar())
        test_ls.append(loss(net(test_features),test_labels).mean().asscalar())
    
    #作图，把想要画的画下来
    d2l.semilogy(range(1, num_epochs + 1), train_ls, 'epochs', 'loss',range(1, num_epochs + 1), test_ls, ['train', 'test'])
    print('L2 norm of w:',net[0].weight.data().norm().asscalar())


In [ ]:
'''过拟合展示'''
fit_and_plot_gluon(0)

In [ ]:
'''权重衰减之后'''
fit_and_plot_gluon(3)

In [ ]:
'''总结：

1.正则化通过为模型损失函数添加惩罚项使学出的模型参数值较小，是应对过拟合的常用手段
2.权重衰减等价于L2范数正则化，通常会使学到的权重参数的元素接近于0
3.权重衰减可以通过Gluon的wd超参数来指定
4.可以定义多个Trainer实例对不同的模型参数使用不同的迭代方法

'''

